In [54]:
from datetime import datetime
import pandas as pd
from datetime import timezone
import pickle

In [55]:
amazon_news = pd.read_pickle("pkls/news_data/amazon_news_data.pkl", compression='infer')
apple_news = pd.read_pickle("pkls/news_data/apple_news_data.pkl", compression='infer')
amazon = "amzn"
apple = "aapl"

In [56]:
import json
config = json.load(open("../configure.json"))
pkls_dir = config['pkls_dir']
start_date = datetime.fromisoformat(config['start_time']).astimezone(tz=timezone.utc).replace(tzinfo=None).timestamp()
end_date = datetime.fromisoformat(config['end_time']).astimezone(tz=timezone.utc).replace(tzinfo=None).timestamp()
chart_pkls_dir = config['chart_pkls']

In [57]:
import datetime

index = 0
row = []
minutes = 1440

form = "%d-%m-%Y %H:%M"
length_of_time_windows = [5, 15, 30, 60, 240, 1440]
full_data_pkls_path = config['full_data_pkls']


def get_time_to_trend(chart, time_interval):
    time_map = {}
    if time_interval == 1440:
        for chart_index, chart_row in chart.iterrows():
            day = datetime.datetime.utcfromtimestamp(chart_row['timestamp'])
            day = (day - datetime.timedelta(hours=day.hour, minutes=day.minute, seconds=day.second,
                                            microseconds=day.microsecond))
            timestamp = day.strftime(form)
            time_map[timestamp] = chart_row['trends']
    else:
        for chart_index, chart_row in chart.iterrows():
            timestamp = datetime.datetime.utcfromtimestamp(chart_row['timestamp']).strftime(form)
            time_map[timestamp] = chart_row['trends']
    return time_map


def handle_compare(time_window, day, time_to_trend):
    if time_window == 1440:
        day = (day - datetime.timedelta(hours=day.hour, minutes=day.minute, seconds=day.second,
                                        microseconds=day.microsecond))
        time_stamp = day.strftime(form)
        while time_stamp not in time_to_trend:
            day += datetime.timedelta(minutes=time_window)
            time_stamp = day.strftime(form)
        return time_stamp
    if time_window == 240:
        day = (day - datetime.timedelta(seconds=day.second,
                                        microseconds=day.microsecond))
        time_stamp = day.strftime(form)
        if day.minute > 30:
            day = day - datetime.timedelta(minutes=day.minute) + datetime.timedelta(minutes=30)
        if day.minute < 30:
            day = day - datetime.timedelta(minutes=day.minute) - datetime.timedelta(minutes=30)
        for i in range(0, 3):
            day -= datetime.timedelta(minutes=60)
            time_stamp = day.strftime(form)
            if time_stamp in time_to_trend:
                return time_stamp
        while time_stamp not in time_to_trend:
            day += datetime.timedelta(minutes=60)
            time_stamp = day.strftime(form)
        return time_stamp
    if time_window == 60:
        day = (day - datetime.timedelta(seconds=day.second,
                                        microseconds=day.microsecond))
        time_stamp = day.strftime(form)
        if day.minute > 30:
            day = day - datetime.timedelta(minutes=day.minute) + datetime.timedelta(minutes=30)
        if day.minute < 30:
            day = day - datetime.timedelta(minutes=day.minute) - datetime.timedelta(minutes=30)
        while time_stamp not in time_to_trend:
            day += datetime.timedelta(minutes=60)
            time_stamp = day.strftime(form)
        return time_stamp
    if time_window == 30:
        day = (day - datetime.timedelta(seconds=day.second,
                                        microseconds=day.microsecond))
        time_stamp = day.strftime(form)
        if day.minute > 30:
            day = day - datetime.timedelta(minutes=day.minute) + datetime.timedelta(minutes=30)
        if day.minute < 30:
            day = day - datetime.timedelta(minutes=day.minute)
        while time_stamp not in time_to_trend:
            day += datetime.timedelta(minutes=30)
            time_stamp = day.strftime(form)
        return time_stamp
    if time_window == 15:
        day = (day - datetime.timedelta(seconds=day.second,
                                        microseconds=day.microsecond))
        time_stamp = day.strftime(form)
        mins = day.minute
        div = mins // 15
        day = day - datetime.timedelta(minutes=day.minute) + datetime.timedelta(minutes=15 * div)
        while time_stamp not in time_to_trend:
            day += datetime.timedelta(minutes=15)
            time_stamp = day.strftime(form)
        return time_stamp
    if time_window == 5:
        day = (day - datetime.timedelta(seconds=day.second,
                                        microseconds=day.microsecond))
        time_stamp = day.strftime(form)
        mins = day.minute
        div = mins // 5
        day = day - datetime.timedelta(minutes=day.minute) + datetime.timedelta(minutes=5 * div)
        while time_stamp not in time_to_trend:
            day += datetime.timedelta(minutes=5)
            time_stamp = day.strftime(form)
        return time_stamp


def get_full_data(news_data, company):
    for time_window in length_of_time_windows:
        if company == amazon:
            chart = pd.read_pickle(chart_pkls_dir + "amazon_stock_chart" + str(time_window) + ".pkl")
        else:
            chart = pd.read_pickle(chart_pkls_dir + "apple_stock_chart" + str(time_window) + ".pkl")
        time_to_trend = get_time_to_trend(chart, time_window)
        news = news_data.copy(deep=True)
        trends = []
        try:
            for index, row in news.iterrows():
                day = datetime.datetime.utcfromtimestamp(row['published_time'])
                timestamp = handle_compare(time_window, day, time_to_trend)
                trends.append(int(time_to_trend[timestamp]))
        except Exception as e:
            print(e)
            print("it fails")
            print(row, index)
        news['trend'] = trends
        news.drop("time_frames", axis=1, inplace=True)
        news.drop("published_time", axis=1, inplace=True)
        if company == amazon:
            with open(full_data_pkls_path + 'amazon_full_data' + str(time_window) + '.pkl', 'wb') as f:
                pickle.dump(news, f)
        if company == apple:
            with open(full_data_pkls_path + 'apple_full_data' + str(time_window) + '.pkl', 'wb') as f:
                pickle.dump(news, f)


get_full_data(amazon_news, amazon)
get_full_data(apple_news, apple)